## Aula 26 - NLP

### Warm up

![](https://media1.tenor.com/images/2c5e2003710c3f8e3d39ebb424b80f0c/tenor.gif?itemid=6169062)

### Expectativas

![](https://everythingbutthebooks.files.wordpress.com/2014/06/gif-rapunzel.gif)

### O que é NLP?

- Campo relativamente novo da computação que combina ML e linguística.
- Principal foco: fazer as máquinas entendam (e até se comuniquem) em linguagem humana. 
- Área de pesquisa/atuação extremamente ampla.
- Biggest consequence: lowering (or complete removal) of the barrier to entry for BI and big data in general: 

> "Google might tell you today what the weather will be tomorrow. But soon enough, you’ll be able to ask your personal data chatbot about customer sentiment today, and how they’ll feel about your brand next week; all while walking down the street."
> https://www.sisense.com/blog/heres-natural-language-processing-future-bi/

<img src="images/nlp-diagram.jpg" width="800"/>

##### Exemplos?

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.



![](images/nlp-fields.png)

                       - Autocomplete: celular, e-mail, pesquisa Google.
                       - Spell checker
                       - Spam detection
                       - Information Retrieval: query do usuário -> produto/documento (Google)
                       - Chatbot
                       - Q&A: pergunta -> resposta (Watson/Jeopardy)
                       - Speech recognition
                       - Machine Translation (Google Translate)

### Os dados: 
Suponha que você é um Cientista de Dados, trabalha em um app de um restaurante e tem reviews de alguns clientes.

*PS: Esse conjunto de dados é composto de reviews de restaurantes e foi modificado a partir do dataset usado no workshop SemEval (International Workshop on Semantic Evaluation) na [edição de 2016](http://alt.qcri.org/semeval2016/task5/).*

In [ ]:
import re
import pickle
import pandas as pd
import en_core_web_sm
import spacy
from spacy.lang.en import English
from collections import Counter
from pycontractions import Contractions
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from utils.confusion_matrix import plot_confusion_matrix
from utils.to_dense import DenseTransformer

pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

In [ ]:
#!python -m spacy download en_core_web_sm
#!pip install pycontractions

In [ ]:
pd.read_csv('data/raw/raw_reviews.csv').sample(5)

### Qual a tarefa??

Como agregar valor ao negócio a partir desses dados?


.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.


### Tarefa: *Sentiment analysis*
Classificar review em positiva ou negativa.

In [ ]:
train_df = pd.read_csv('data/processed/train.csv')
test_df = pd.read_csv('data/processed/test.csv')

train_df.head()

### As ferramentas
- Principal biblioteca atualmente para trabalhar com NLP: *Spacy*. 

* O datacamp lançou recentemente um [curso](https://campus.datacamp.com/courses/advanced-nlp-with-spacy) bem legal sobre ele que vale a pena dar uma conferida já que hoje, focaremos no "básico".

- Outra lib bastante usada (principal biblioteca até alguns meses atrás): [NLTK](https://www.nltk.org/), .

- Outras iniciativas famosas com deep learning: [AllenNLP](https://allennlp.org/) e o [StanfordNLP](https://stanfordnlp.github.io/stanfordnlp/), que são capazes de atingir o estado da arte de muitas aplicações.


#### Spacy: 
- Conceito de objeto (comumente chamada de 'nlp') que contém todo o pipeline de processamento, além de outras regras específicas de uma certa língua.

In [ ]:
#from spacy.lang.en import English
nlp = English()

### Como a máquina vai "entender" os textos?

[![](http://img.youtube.com/vi/d4gGtcobq8M/0.jpg)](http://www.youtube.com/watch?v=d4gGtcobq8M "")


                    **Patterns, Disambiguation (context), Stopwords, Tokens, Bag-of-words, TF, TF-IDF**

**Pipeline NLP**
<img src="images/nlp-pipe.png" width="500"/>

#### Patterns
Em alguns casos, pode ser interessante substituir certas partes em textos fixos, como horários, quantidades, emails.

Exemplo:

In [ ]:
re.sub(u'[a-z0-9\_\.\-]*@[a-z0-9\_\.\-]*', 'EMAIL_ADDR', u'hi there thais.neubauer@gmail.com!',flags=re.UNICODE)

#### Tokenization
- Segmentação de sentenças em "palavras".
- Análise léxica: eliminação de caracteres de pontuação e outros caracteres "especiais" - não alfa-numéricos (em alguns casos, excluindo até mesmo os numéricos e acentuação).

Exemplos de potenciais problemas:

<img src="images/nlp-tokenizing_problems.png" width="600"/>

In [ ]:
doc = nlp("We went around 9:30 on a Friday...")
tokens = [token.text for token in doc]
tokens

In [ ]:
doc = nlp("It's also attached to Angel's Share, which is a cool, more romantic bar...")
tokens = [token.text for token in doc]
tokens

* Exercício: Crie o objeto nlp para português e imprima o texto do primeiro token

In [ ]:
# Import the Portuguese language class
from spacy.lang.___ import ________

# Create the nlp object
nlp_pt = ___________

# Process a text
doc = nlp_pt("Isso é uma sentença")

# Select the first token
first_token = _______

# Print the first token's text
print(first_token._______)

### Pré-processamento

#### Removing stopwords (and maybe other tokens)
- Para muitas tarefas de NLP, é bom prestar atenção nas chamadas *stopwords*, que são as palavras muito comuns que aparecem no texto e que, por isso, não tem potencial para contribuir para a caracterização do conteúdo presente no texto. 

- Nessa lista geramente estão: artigos definidos e indefinidos, preposições, pronomes, numerais, conjunções e advérbios. 

-> IMPORTANTE: Além das palavras pertencentes a essas classes gramaticais, podem entrar na lista as palavras muito comuns dentro do contexto referente aos documentos do corpus.

Lei de Zipf e proposta de cortes de Luhn:

<img src="images/nlp-freq_import.png" width="500"/>

In [ ]:
#from collections import Counter
#splita as frases por palavras (espaco incluido) e soma elas
Counter(sum(train_df['text'].str.lower().str.split(r'[\W\s]+').tolist(), [])).most_common(10)

***

O que essa regex está fazendo é basicamente convertendo todo o texto para minúsculo e depois 'splitando' em tokens sempre que o texto encontra um caracter em brando (\s) **ou** sempre que encontra um caracter que **não** é alfanumérico ([^a-zA-Z0-9]), representado pelo \W

Depois, ela pega as 10 ocorrências mais comumns.

***

No spacy já existe uma lista de stopwords comuns, que podemos ver assim:

In [ ]:
nlp = spacy.load('en_core_web_sm')

en_stopwords = sorted([token.text for token in nlp.vocab if token.is_stop])
en_stopwords[155:161]

* Exercício:
Explore um pouco a lista e analise as que você acha que podem ser excluídas.

In [ ]:
list_excl = ['cannot', 'go', 'off']

for w in list_excl:
    nlp.vocab[w].is_stop = False

Para verificarmos quais palavras excluímos da lista de stopwords:

In [ ]:
' -- '.join([w for w in en_stopwords if not nlp.vocab[w].is_stop])

- Além de excluir, podemos também incluir, pois, como dito no início da conversa sobre *stopwords*, podem entrar na lista as palavras muito comuns dentro do contexto referente aos documentos do corpus.
- Outra forma de **definir stopwords** é **verificando a frequência das palavras no corpus**, ou seja, contando a frequência de cada token no *corpus*.

#### Stemming
Normalize words into its base or root form, using common prefixes and suffixes.

Exemplo: affectation, affects, affections, affected, affection, affecting -> affect.

Problemas: boiando -> boi, factual -> fact (suffixe "ual"), equal -> eq (suffixe "ual").

#### Lemmatization
Also maps several similar words into one common root, but groups together different inflected forms of a word to its *lemma* (dictionary needed), which is a proper word. 

Exemplo: gone, going, went -> go

#### Part-Of-Speech (POS) Tags
Grammatical type of the word: noun, verb, adjective, adverb, pronoun, preposition, conjunction, determiner, exclamation.

Exemplo: 

<img src="images/nlp-pos.png" width="300" align="left"/>

##### Objeto "Token" - Spacy

É possível carregar alguns modelos pré treinados no Spacy para prever atributos linguísticos, como:

- POS tags (classificação gramátical - Apple companhia vs apple fruta)
- Nomeação de Entidades (Apple companhia, Photograph Nickelback album)


Tais modelos estão divididos em pacotes que precisam ser baixados, como o 'en_core_web_sm', pacote com vários modelos treinados em inglês.


O objeto Token é um objeto que possui vários atributos, inclusive o "pos_" . Outros atributos importantes/legais:

<img src="images/spacy.png" width="700"/>

In [ ]:
nlp = English()
nlp = spacy.load('en_core_web_sm')

doc = nlp("She ate the pizza")
for token in doc:
    print(token.text, token.pos_)

In [ ]:
doc = nlp("I heard Photograph from Nicklelback yesterday !")
for ent in doc.ents:
    print(ent.text, ent.label_)

* Exercício: Se usarmos .lower() no texto abaixo, como ficam as entidades?

#### Normalização:

- Além do que já comentamos até aqui, outras diferentes formas de escrever a mesma coisa podem "confundir" os algoritmos. 

- Transformar o texto todo em minúsculo e retirar acentos, pro exemplo, são bastante comuns. 

CUIDADO pra essas etapas não atrapalharem outras, como identificação de entidades!!!


##### Contractions 
Outro exemplo de normalização interessante na língua inglesa é a expansão de contrações: I'm -> I am, We're -> we are etc.

Para expansão de contractions, usaremos o [PyContractions](https://pypi.org/project/pycontractions/) e especificaremos o modelo da api gensim.downloader:

In [ ]:
cont = Contractions(api_key="glove-twitter-100")
list(cont.expand_texts(["We're all so happy!! Can't believe this!"]))

#### Etapas do pré-processamento:


1. Remove entidades (remove_ents);
2. Expande constrações (expand_contractions);
3. Transforme tudo para minúscula;
4. Remove stopwords e pontuações (remove_stop_and_punct).

IMPORTANTE: Atenção na ordem!!!

- Exercício: Crie uma função ```preprocess``` que recebe qualquer string como entrada e faz todas as etapas do pré-processamento.

In [ ]:
def remove_ents(text):
    doc = nlp(text)
    for ent in doc.ents:
        text = text.replace(ent.text, '')
    return text


def expand_contractions(text):
    return list(________________)[0]

        
def remove_stop_and_punct(text):
    doc = nlp(text.lower())
    tokens = []
    for token in doc:
        if _________________:
            continue
        tokens.append(token.text)
    return ' '.join(tokens)

In [ ]:
def preprocess(original_text):
    new_text = remove_ents(original_text)
    new_text = expand_contractions(new_text)
    if len(new_text) == 0:
        return new_text 
    return remove_stop_and_punct(new_text)

In [ ]:
#Ja fizemos isso neste notebook, mas aqui teriamos que carregar esses objetos se já não estivéssemos feito ainda, 
#pois as funções estão usando-os. Porém, atenção: carregamos E alteramos o en_core_web_sm quando exluímos algumas 
#palavras da lista de stopwords, então se carregarmos novamente, temos que excluir as stopwords novamente também

#nlp = English()
#nlp = spacy.load('en_core_web_sm') 

In [ ]:
assert(preprocess("It's also not attached to Angel's Share, which is a cool, more romantic bar...") 
       == 'not attached cool romantic bar')

- Exercício: Aplique sua função preprocess à coluna text (dos dois dataframes, train_df e test_df), criando uma coluna nova norm_text, que será usada para treinarmos um modelo de análise de sentimento. Dica: Use o apply do pandas!

In [ ]:
train_df['norm_text'] = train_df['text'].apply(preprocess)
train_df.to_pickle('data/processed/train_preprocessed.pickle')
print("Train:\n")
display(train_df[['text','norm_text','polarity']].sample(n=5))

In [ ]:
test_df['norm_text'] = test_df['text'].apply(preprocess)
test_df.to_pickle('data/processed/train_preprocessed.pickle')
print("Test:\n")
display(test_df[['text','norm_text','polarity']].sample(n=5))

### Feature engineering

#### Bag of words

<img src="images/nlp-bow.png" width="400"/>

- Com o Scikit Learn: [CountVectorizer](https://scikit-learn.org/0.19/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [ ]:
examples_for_bow = [
    'camisa preta botao botao botao',
    'botao feito linha preta',
    'considera-se caro preco botao camisa botao',
    'linha costurar botão mesma camisa',
    'costurar linha camisa mesma botao'
]

In [ ]:
cv = CountVectorizer(strip_accents='unicode', binary=True)
bow_matrix = cv.fit_transform(examples_for_bow)
columns = [token[0] for token in sorted(cv.vocabulary_.items(), key=lambda item: item[1])]
pd.DataFrame(bow_matrix.todense(), columns=columns)

- Exercício: Use o parâmetro 'max_features' do CountVectorizer para diminuir a quantidade de dimensões e veja a diferença na matriz resultante:

#### TF: E se usássemos a quantidade de vezes que o token ocorre?
<img src="images/nlp-bow2.png" width="800"/>

- Exercício: usando o [CountVectorizer](https://scikit-learn.org/0.19/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) mesmo, como obter a matriz TF?

#### TF-IDF
<img src="images/nlp-tfidf.png" width="800"/>

- Exercício: use o [TfidfVectorizer](https://scikit-learn.org/0.19/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) para vetorizar os mesmos dados dos outros vetorizadores que estamos usando.

#### N-grams
<img src="images/nlp-ngrams.png" width="500"/>

### Modelo

Os dados estão prontos para "enviar" pro modelo?

In [ ]:
train_df.sample(3)

In [ ]:
test_df.sample(3)


.

.

.

.

.

.

.

.

.

.

.

.

.

Não, ne? Próximos passos:
1. Target de string pra inteiro.
2. Separar X_train, y_train, X_test, y_test

- Exercício: transforme a coluna polarity em inteiro (0 se "negative" e 1 se "positive") e verifique em um sample o resultado.

In [ ]:
train_df['y'] = ____________
train_df.sample(3)

In [ ]:
test_df['y'] = ____________
test_df.sample(3)

In [ ]:
X_train = train_df['norm_text'].values
y_train = train_df['y'].values
X_test = test_df['norm_text'].values
y_test = test_df['y'].values

- Exercício: monte um [Pipeline](https://scikit-learn.org/0.19/modules/generated/sklearn.pipeline.Pipeline.html) com o CountVectorizer binário e LogisticRegression.

In [ ]:
steps = [
    ('vect', __________),
    ('clf', ___________)
]

pipeline = Pipeline(steps)

In [ ]:
sentiment_analyzer = pipeline.fit(X_train, y_train)

In [ ]:
sentiment_analyzer.________(["It's also attached to Angel's Share, which is a cool, more romantic bar..."])

In [ ]:
sentiment_analyzer.________(["It's also attached to Angel's Share, which is a cool, more romantic bar..."])

### Avaliação

- Exercício: Avalie o resultado com a *classification_report* e com a *plot_confusion_matrix*

In [ ]:
y_pred = ____________________

In [ ]:
class_names = ['negative', 'positive']
classification_report(______, ______, target_names=class_names)

In [ ]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(________, ________);

# Plot normalized confusion matrix
